In [38]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

In [39]:
image_directory = '/home/areisdorf/code/areisdorf45/solar_panel_segmentation/notebooks/simple_data/single_img/images/austin1.tif'

In [42]:
img = cv2.imread(image_directory)

array([[[121, 127, 123],
        [123, 128, 124],
        [121, 126, 124],
        ...,
        [ 90,  99, 103],
        [ 99, 110, 113],
        [ 95, 108, 108]],

       [[124, 129, 125],
        [127, 131, 126],
        [123, 127, 124],
        ...,
        [ 93, 102, 105],
        [ 96, 110, 113],
        [ 94, 108, 108]],

       [[124, 128, 124],
        [123, 127, 124],
        [123, 126, 124],
        ...,
        [ 95, 104, 106],
        [ 97, 111, 115],
        [ 95, 109, 111]],

       ...,

       [[ 45,  47,  38],
        [ 27,  31,  27],
        [ 37,  38,  34],
        ...,
        [ 84,  90,  93],
        [ 34,  38,  43],
        [ 32,  34,  43]],

       [[ 43,  44,  36],
        [ 45,  44,  37],
        [ 54,  50,  44],
        ...,
        [ 32,  39,  44],
        [ 37,  40,  43],
        [ 30,  35,  41]],

       [[ 44,  44,  37],
        [ 64,  59,  50],
        [ 50,  48,  42],
        ...,
        [ 34,  39,  41],
        [ 33,  37,  40],
        [ 73,  75,  83]]

In [49]:
size = 256
overlap = 4

indices = np.arange(0, (5000 - (size - overlap)), size - overlap)
#output_loc = '/home/areisdorf/code/areisdorf45/solar_panel_segmentation/notebooks/simple_data/simple_pred'

for x in indices:
    for y in indices:
        split = img[x:(x + size), y:(y + size), :]
        array = np.array(split)
        cv2.imwrite('simple_slice.png', array )

In [50]:
def image_split(dictionary, overlap, size):
    
   indices = np.arange(0, (5000 - (size - overlap)), size - overlap)
   files = os.listdir(dictionary)
   
   for file in files:
        img = io.imread(f'{dictionary}/{file}')
        for x in indices:
            for y in indices:
                split = img[x:(x + size), y:(y + size), :]
                array = np.array(split)
                image.imsave(f'{dictionary}/../simple_slice/{file}_{x}_{y}.png', array)

In [65]:
def mask_split(dictionary, overlap, size):
    
   indices = np.arange(0, (5000 - (size - overlap)), size - overlap)
   files = os.listdir(dictionary)
   
   for file in files:
        img = io.imread(f'{dictionary}/{file}')
        for x in indices:
            for y in indices:
                split = img[x:(x + size), y:(y + size), :]
                array = np.array(split)
                image.imsave(f'{dictionary}/../mask_slice/{file}_{x}_{y}.png', array)

In [69]:
mask_split('/home/areisdorf/code/areisdorf45/solar_panel_segmentation/notebooks/simple_data/single_img/gt', 4, 256)

NameError: name 'io' is not defined

In [59]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.utils import image_dataset_from_directory

In [57]:
def simple_unet_model(image):

    #Input
    height = 256
    width = 256
    channels = 3
    inputs = Input((height, width, channels))
    inputs = Lambda(lambda x: x / 255)(inputs) #Normalize the pixels by dividing by 255

    #Encoder where we are extracting the features
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    convolution1 = Dropout(0.1)(convolution1) #other option is to do batch normalization
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution1)
    pooling1 = MaxPooling2D((2, 2))(convolution1)
    
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(pooling1)
    convolution2 = Dropout(0.1)(convolution2)
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(convolution2)
    pooling2 = MaxPooling2D((2, 2))(convolution2)
     
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(pooling2)
    convolution3 = Dropout(0.2)(convolution3)
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(convolution3)
    pooling3 = MaxPooling2D((2, 2))(convolution3)
     
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(pooling3)
    convolution4 = Dropout(0.2)(convolution4)
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(convolution4)
    pooling4 = MaxPooling2D(pool_size=(2, 2))(convolution4)
     
    convolution5 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(pooling4)
    convolution5 = Dropout(0.3)(convolution5)
    convolution5 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(convolution5)
    
    #Decoder where we are indicating to the model the precise location of the features 
    transconv6 = Conv2DTranspose(128, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution5)
    transconv6 = concatenate([transconv6, convolution4])
    convolution6 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(transconv6)
    convolution6 = Dropout(0.2)(convolution6)
    convolution6 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(convolution6)
     
    transconv7 = Conv2DTranspose(64, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution6)
    transconv7 = concatenate([transconv7, convolution3])
    convolution7 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(transconv7)
    convolution7 = Dropout(0.2)(convolution7)
    convolution7 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(convolution7)
     
    transconv8 = Conv2DTranspose(32, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution7)
    transconv8 = concatenate([transconv8, convolution2])
    convolution8 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(transconv8)
    convolution8 = Dropout(0.1)(convolution8)
    convolution8 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(convolution8)
     
    transconv9 = Conv2DTranspose(16, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution8)
    transconv9 = concatenate([transconv9, convolution1], axis=3)
    convolution9 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(transconv9)
    convolution9 = Dropout(0.1)(convolution9)
    convolution9 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution9)
     
    outputs = Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(convolution9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    jaccard = MeanIoU(2)
    
    #loss options include: binary_crossentropy, IoU Loss (Jaccard Index), dice coefficient
    model.compile(optimizer='adam', loss=jaccard, metrics=['accuracy']) 
    
    model.summary()
    
    return model

In [58]:
simple_unet_model(img)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_3 (Conv2D)              (None, 256, 256, 16  448         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 dropout_2 (Dropout)            (None, 256, 256, 16  0           ['conv2d_3[1][0]']               
                                )                                                             

In [60]:
sliced_image_directory = '/home/areisdorf/code/areisdorf45/solar_panel_segmentation/notebooks/simple_data/single_img/simple_slice'

In [63]:
training_images = image_dataset_from_directory(sliced_image_directory)

Found 361 files belonging to 1 classes.


In [64]:
type(training_images)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [ ]:
    #Encoder where we are extracting the features
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    
    x = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(x)
    
    x = MaxPooling2D((2, 2))(x)
      
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    convolution1 = BatchNormalization()(convolution1) #other option is to do batch normalization
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution1)
    pooling1 = MaxPooling2D((2, 2))(convolution1)
    
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(pooling1)
    convolution2 = BatchNormalization()(convolution2)
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(convolution2)
    pooling2 = MaxPooling2D((2, 2))(convolution2)
     
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(pooling2)
    convolution3 = BatchNormalization()(convolution3)
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(convolution3)
    pooling3 = MaxPooling2D((2, 2))(convolution3)
     
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(pooling3)
    convolution4 = BatchNormalization()(convolution4)
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(convolution4)
    pooling4 = MaxPooling2D(pool_size=(2, 2))(convolution4)